In [597]:
import matplotlib.pyplot as plt
import pandas as pd

from scipy.io import loadmat

In [598]:
import numpy as np
dataset = loadmat('hw04_data.mat')

In [599]:
import torch
import pandas as pd

# Represents a layer in the neural network. Takes in input dimension, output dimension 
# and activation function of the layer, which is set to Identity as default.
class Layer(torch.nn.Module):

    # Constructor for the Layer class.
    def __init__(self, inputDim, outputDim, activation = "Identity"):
        
        super(Layer, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim

        # Depending on the string input representing which activation function to use
        # correctly assigns the activation.
        if (activation == "ReLU"):
            self.activation = torch.nn.ReLU()
        elif (activation == "Tanh"):
            self.activation = torch.nn.Tanh()
        elif (activation == "Identity"):
            self.activation = torch.nn.Identity()
        elif (activation == "Sigmoid"):
            self.activation = torch.nn.Sigmoid()

        # How to find the total weighted input
        self.l1 = torch.nn.Linear(self.inputDim, self.outputDim)

        
    # Signature: input vector X -> activation/output
    # Combines the weights and activations of the previous layer, and applies an activation function
    def forward(self, x):
        entry = self.l1(x)
        output = self.activation(entry)
        return output



In [600]:
# Represents a complete neural network with multiple layers. Takes in a list of layers.
class NN:

    # Constructor for the NN class
    def __init__(self, layers):
        
        super(NN, self).__init__()        
        self.layers = layers
        param_list = []
        for layer in layers:
            param_list += list(layer.parameters())
        self.parameters = torch.nn.ParameterList(param_list)

    # Signature: Input data X -> Prediction from NN
    # Passes the data through the neural network and returns a prediction
    def forward(self, x): 
        entry = x
        for layer in self.layers:
            entry = layer.forward(entry)
        return entry
             
    
    # Signature:
    # 
    def score(self, x_test, y_test):
        x_test_tensor = torch.FloatTensor(X_test.values)
        test_results = model.forward(x_test_tensor)
        results = [torch.argmax(res).item() for res in test_results]
        
        total = 0
        for i in range(len(results)):
            if results[i] == y_test[i]:
                total += 1

        return total/len(y_test)

In [601]:
# signature: number of layers, layer sizes for each layer, string representing activation function -> Neural Network
# constructs the neural network
def nn(n, sl, activation):
    layers = [Layer(sl[i], sl[i+1], activation) for i in range(0, len(sl) - 1)]
    return NN(layers)

In [602]:
# signature: NN, nxm dataset, nxp dataset, rxs dataset, rxt dataset -> modifies the model
# training the neural network
def train_nn(model, x_train, y_train, x_test, y_test):

    criterion = torch.nn.CrossEntropyLoss()
    learning_rate = 1e-4
    optimizer = torch.optim.Adam(model.parameters, lr=learning_rate)

    for t in range(5000):
        # Forward pass: compute predicted y by passing x to the model.
        x_train_tensor = torch.FloatTensor(x_train)
        y_pred = model.forward(x_train_tensor)

        # Compute and print loss.
        y_train_tensor = y_train
        loss = criterion(y_pred, y_train_tensor)
        # if t % 100 == 99:
        #     print(t, loss.item())

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). 
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()

In [603]:
# Constructs and runs the neural network with n layers and given sizes of each layer with activation function on the given data
def feed_forward_nn(n, sl, data, activation):

    # split the data into training and testing
    X_trn = data["X_trn"]
    y_trn = data["y_trn"]
    X_tst = data["X_tst"]
    y_tst = data["y_tst"]
    X_trn_torch = torch.FloatTensor(X_trn)

    # create the model based on the desired number of layers and nodes at each layer
    sl.insert(0, X_trn.shape[1])
    sl.append(2)
    model = nn(n, sl, activation)

    # adjust the training y dataset to change -1 to 0 to better handle the data and make it a tensor
    y_updated = []
    for item in y_trn:
        if item[0] == 1:
            y_updated.append([1])
        else:
            y_updated.append([0])
    y_trn_torch = torch.LongTensor(np.array(y_updated)).reshape(-1)

    #y_trn_torch = torch.LongTensor(y_trn).reshape(-1)
    X_tst_torch = torch.FloatTensor(X_tst)
    y_tst_torch = torch.LongTensor(y_tst).reshape(-1)

    # train the model
    train_nn(model, X_trn_torch, y_trn_torch, X_tst_torch, y_tst_torch)

    # print out the parameters
    print("Final neural network layers:\n")
    for parameter in model.parameters:
        print(parameter)

    # print out the activation of the final layer
    results = model.forward(X_tst_torch)
    tst_results = []
    for row in results:
        if row[0] > row[1]:
            tst_results.append([-1])
        else:
            tst_results.append([1])

    tst_results = np.array(tst_results) 
    print("\nPrediction from testing set:\n")
    print(tst_results)

    print("\nAccuracy:")
    print(np.sum(tst_results == y_tst) / len(y_tst))

    return tst_results


In [605]:
# Example: apply the feed forward neural net on the given dataset
feed_forward = feed_forward_nn(3, [2, 50, 25], dataset, "ReLU")

Final neural network layers:

Parameter containing:
tensor([[ 0.6943,  0.6859],
        [ 0.3550, -0.8490]], requires_grad=True)
Parameter containing:
tensor([-0.0144,  0.5951], requires_grad=True)
Parameter containing:
tensor([[ 0.8563, -0.4860],
        [ 0.0702,  0.4182],
        [-0.3972, -0.4066],
        [ 0.8415, -0.1295],
        [-0.6129,  0.3872],
        [ 0.7006, -0.4364],
        [ 0.8387,  0.2803],
        [-0.6094, -0.6769],
        [ 0.5045, -0.1216],
        [ 0.7174, -0.3226],
        [ 0.9137, -0.2464],
        [-0.1859, -0.1894],
        [-0.3565, -0.3899],
        [ 0.0915, -0.8014],
        [-0.8030,  0.5757],
        [ 0.6872,  0.1325],
        [-0.7818,  0.8162],
        [-0.4297, -0.4235],
        [ 0.5602,  0.2475],
        [ 0.4564,  0.3905],
        [-0.5725, -0.5824],
        [-0.1456,  0.8445],
        [ 0.5297,  0.5293],
        [-0.1233,  0.4787],
        [ 0.5169, -0.6596],
        [-0.6245, -0.2269],
        [ 0.2338, -0.6243],
        [ 0.8019, -0.625